# 第二章、向量数据库的介绍及使用



## 一、向量数据库简介

向量数据库是用于高效计算和管理大量向量数据的解决方案。向量数据库是一种专门用于存储和检索向量数据（embedding）的数据库系统。它与传统的基于关系模型的数据库不同，它主要关注的是向量数据的特性和相似性。

在向量数据库中，数据被表示为向量形式，每个向量代表一个数据项。这些向量可以是数字、文本、图像或其他类型的数据。向量数据库使用高效的索引和查询算法来加速向量数据的存储和检索过程。

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

In [1]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

In [2]:
# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env fileopenai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key  = os.environ['OPENAI_API_KEY']
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

In [3]:
# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)


# 定义 Embeddings
embedding = OpenAIEmbeddings() 
# embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
# embedding = ZhipuAIEmbeddings()

In [4]:
persist_directory = '../../data_base/vector_db/chroma'

In [20]:
!rm -rf '../../data_base/vector_db/chroma'  # 删除旧的数据库文件（如果文件夹中有文件的话），window电脑请手动删除

## 二、构建 Chroma 向量库

In [21]:
vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [22]:
vectordb.persist()

大家也可以直接载入已经构建好的向量库

In [5]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [6]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1120


## 三、通过向量数据库检索

### 3.1 相似度检索

In [7]:
question="什么是机器学习"

In [8]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [9]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
导，同时也能体会到这三门数学课在机器学习上碰撞产生的“数学之美”。
1.1
引言
本节以概念理解为主，在此对“算法”和“模型”作补充说明。“算法”是指从数据中学得“模型”的具
体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，
通常是具体的函数或者可抽象地看作为函数，例如一元线性回归算法产出的模型即为形如 f(x) = wx + b
的一元一次函数。
--------------
检索到的第1个内容: 
模型：机器学习的一般流程如下：首先收集若干样本（假设此时有 100 个），然后将其分为训练样本
（80 个）和测试样本（20 个），其中 80 个训练样本构成的集合称为“训练集”，20 个测试样本构成的集合
称为“测试集”，接着选用某个机器学习算法，让其在训练集上进行“学习”（或称为“训练”），然后产出
得到“模型”（或称为“学习器”），最后用测试集来测试模型的效果。执行以上流程时，表示我们已经默
--------------
检索到的第2个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》
←_←
第 1 章
绪论
本章作为“西瓜书”的开篇，主要讲解什么是机器学习以及机器学习的相关数学符号，为后续内容作
铺垫，并未涉及复杂的算法理论，因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可。此外，在
阅读本章前建议先阅读西瓜书目录前页的《主要符号表》，它能解答在阅读“西瓜书”过程中产生的大部
分对数学符号的疑惑。
本章也作为
--------------


### 3.2 MMR 检索

如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。


核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [10]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [11]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
导，同时也能体会到这三门数学课在机器学习上碰撞产生的“数学之美”。
1.1
引言
本节以概念理解为主，在此对“算法”和“模型”作补充说明。“算法”是指从数据中学得“模型”的具
体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，
通常是具体的函数或者可抽象地看作为函数，例如一元线性回归算法产出的模型即为形如 f(x) = wx + b
的一元一次函数。
--------------
MMR 检索到的第1个内容: 
而人工智能的基本挑战是

学习在不确定的情况下做出好的决策

这边我举个例子

比如你想让一个小孩学会走路

他就需要通过不断尝试来发现

怎么走比较好

怎么走比较快

强化学习的交互过程可以通过这张图来表示

强化学习由智能体和环境两部分组成

在强化学习过程中

智能体与环境一直在交互

智能体在环境中获取某个状态后

它会利用刚刚的状态输出一个动作

这个动作也被称为决策

然后这个动作会
--------------
MMR 检索到的第2个内容: 
与基础语言模型不同，指令微调 LLM 通过专门的训练，可以更好地理解并遵循指令。举个例子，当询问“法国的首都是什么？”时，这类模型很可能直接回答“法国的首都是巴黎”。指令微调 LLM 的训练通常基于预训练语言模型，先在大规模文本数据上进行预训练，掌握语言的基本规律。在此基础上进行进一步的训练与微调（finetune），输入是指令，输出是对这些指令的正确回复。有时还会采用RLHF（reinforce
--------------


可以看到内容有了更多的差异。

## 四、构造检索式问答连

我们已经可以通过向量数据库找到最相关的内容了，接下来我们可以让 LLM 来用这些相关的内容回答我们的问题。

### 4.1 直接询问 LLM

基于 LangChain，我们可以构造一个使用 LLM 进行问答的检索式问答链，这是一种通过检索步骤进行问答的方法。我们可以通过传入一个语言模型和一个向量数据库来创建它作为检索器。然后，我们可以用问题作为查询调用它，得到一个答案。

In [12]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModel

In [13]:
llm = OpenAI(temperature=0)

In [ ]:
# 可以使用 HuggingFacePipeline 本地搭建大语言模型
model_id = 'THUDM/chatglm2-6b-int4' # 采用 int 量化后的模型可以节省硬盘占用以及实时量化所需的运算资源
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().quantize(4).cuda()
model = model.eval()
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

llm = HuggingFacePipeline(pipeline=pipe)

In [18]:
llm = ZhipuAILLM(model="chatglm_std", temperature=0)

In [14]:
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [15]:
# 可以以该方式进行检索问答
question = "本知识库主要包含什么内容"
result = qa_chain({"query": question})
print(f"大语言模型的回答为：{result['result']}")

大语言模型的回答为： 这个知识库主要包含了一些关于强化学习的基础知识，传统的强化学习算法，适用强化学习算法的解决方法，简单生动的例子，专业的公式推导和分析，注解，观念词，习题和面试题，代码实战，以及如何高效学习的指导。


### 4.2 结合 prompt 提问

对于 LLM 来说，prompt 可以让更好的发挥大模型的能力。


我们首先定义了一个提示模板。它包含一些关于如何使用下面的上下文片段的说明，然后有一个上下文变量的占位符。

In [16]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [17]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [20]:
question = " 2025 年大语言模型效果最好的是哪个模型"

In [21]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

LLM 对问题的回答：2025年，大语言模型的效果将取决于各种技术的发展情况。目前，OpenAI 的 GPT-3 模型是最先进的大语言模型，它的效果非常出色。但是，随着技术的发展，2025 年可能会有更先进的模型出现，效果更好。感谢您的提问！


这里因为没有对应的信息，所以大语言模型只能回答不知道。您可以将知识库的内容调整为大语言模型综述的内容重新进行尝试。

In [22]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='在本模块，我们将与读者分享提升大语言模型应用效果的各种技巧和最佳实践。书中内容涵盖广泛，包括软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。我们衷心希望该课程能激发读者的想象力，开发出更出色的语言模型应用。\n\n随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前，有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如果你以“法国的首都是什么”为 Prompt ，则基础 LLM 可能会根据互联网上的文章，将回答预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。' metadata={'source': '../knowledge_base/prompt_engineering/1. 简介 Introduction.md'}


这种方法非常好，因为它只涉及对语言模型的一次调用。然而，它也有局限性，即如果文档太多，可能无法将它们全部适配到上下文窗口中。

langchain 提供了几种不同的处理文档的方法：

|     类型      |                                定义/区别                                |                              优点                              |                              缺点                              |
|-------------|---------------------------------------------------------------------|----------------------------------------------------------------|----------------------------------------------------------------|
|   Stuff     | 将整个文本内容一次性输入给大模型进行处理。                               | - 只调用大模型一次，节省计算资源和时间。<br>- 上下文信息完整，有助于理解整体语义。<br>- 适用于处理较短的文本内容。 | - 不适用于处理较长的文本内容，可能导致模型过载。                |
|   Refine    | 通过多次调用大模型逐步改进文本质量，进行多次迭代优化。                          | - 可以在多次迭代中逐步改进文本质量。<br>- 适用于需要进行多次迭代优化的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的文本质量。<br>- 不适用于实时性要求较高的场景。 |
| Map reduce  | 将大模型应用于每个文档，并将输出作为新文档传递给另一个模型，最终得到单个输出。               | - 可以对多个文档进行并行处理，提高处理效率。<br>- 可以通过多次迭代处理实现优化。<br>- 适用于需要对多个文档进行处理和合并的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的结果。<br>- 不适用于处理单个文档的场景。 |
| Map re-rank | 在每个文档上运行初始提示，为答案给出一个分数，返回得分最高的响应。                        | - 可以根据置信度对文档进行排序和选择，提高结果的准确性。<br>- 可以提供更可靠的答案。<br>- 适用于需要根据置信度对文档进行排序和选择的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要对多个文档进行评分和排序。<br>- 不适用于不需要对文档进行排序和选择的场景。 |

我们可以根据需要配置 chain_type 的参数，选择对应的处理方式。如：
```
RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
```